# 图和函数简介

## 概述

本指南深入TensorFlow和Keras的知识面，看看TensorFlow是如何工作的。如果你想立即开始学习Keras，请参阅我们的Keras指南部分。

在本指南中，您将看到TensorFlow如何允许您对代码进行简单更改以获得计算图，以及如何存储和表示计算图，以及如何使用它们来加速和导出模型。

注意:对于那些只熟悉TensorFlow 1.x的同学，本指南演示了一个非常不同的计算图视图。
这是一个简短的介绍;有关这些概念的完整介绍，请参阅[the tf.function guide](https://tensorflow.google.cn/guide/function)。

### 计算图是什么?

在前三部分指南中，您已经看到TensorFlow即刻运行（Excute Eagerly）。这意味着TensorFlow操作由Python执行，一个操作接着一个操作的执行，然后将结果返回给Python。Eager TensorFlow利用了gpu的优势，允许您在gpu和TPUs上放置变量、张量，甚至操作。它也很容易调试。

对于某些用户，您可能永远不需要或不想离开Python。

但是，在Python中运行TensorFlow一个接一个的操作会阻止大量其他可用的加速。如果你能从Python中提取出张量计算，你就能把它们做成图。

**计算图是一种数据结构，它包含一组操作对象（表示计算单位）和张量对象（表示在操作之间流动的数据）。它们被定义为tf.Graph context。由于这些图是数据结构，所以可以在不使用原始Python代码的情况下保存、运行和恢复它们。**

这就是一个简单的两层图在TensorBoard中显示的样子。

![a two-layer tensorflow graph](	
https://storage.cloud.google.com/tensorflow.org/images/two-layer-network.png)

### 计算图的好处

有了计算图，您就有了很大的灵活性。您可以在没有Python解释器的环境中使用TensorFlow图，比如移动应用程序、嵌入式设备和后端服务器。当从Python导出模型时，TensorFlow使用计算图作为保存模型的格式。

计算图也很容易优化，允许编译器做像这样的转换:

* 通过在计算中折叠常数节点(“常数折叠”)来静态地推断张量的值。
* 计算的独立子部分，并在线程或设备之间分割它们。
* 通过消除公共子表达式来简化算术运算。


有一个完整的优化系统Grappler执行这个和其他加速。

简而言之，计算图非常有用，它可以让您的TensorFlow快速运行、并行运行并在多个设备上高效运行。

但是，为了方便起见，您仍然希望用Python定义我们的机器学习模型(或其他计算)，然后在需要时自动构造图。

## 跟踪图（Tracing graphs）

在TensorFlow中创建图形的方法是使用tf.function，可以作为直接调用，也可以作为装饰器。

In [ ]:
import tensorflow as tf
import timeit
from datetime import datetime

In [ ]:
# 定义Python函数
def function_to_get_faster(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# 创建一个包含计算图的函数对象
a_function_that_uses_a_graph = tf.function(function_to_get_faster)

# 创建一些张量
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

# It just works!
a_function_that_uses_a_graph(x1, y1, b1).numpy()

`tf.function`-ized 函数是Python可调用函数，其工作原理与Python的等价函数相同。它们有一个特定的类(python.eager.def_function.Function)，但对您来说，它们就像未跟踪版本一样。

`tf.function` 递归地跟踪它调用的任何Python函数。

In [ ]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# 使用装饰器
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# 函数调用将创建包括inner_function()和outer_function()的计算图
outer_function(tf.constant([[1.0, 2.0]])).numpy()

如果你用过TensorFlow 1.x，您将注意到在任何时候都不需要定义占位符或tf.Sesssion。

### 流控制和副作用（Flow control and side effects）

流程控制和循环通常通过默认方法tf.autograph转换为Tensorflow。Autograph使用了一组方法，包括标准化循环构造、展开和AST（Abstract Syntax Trees，抽象语法树）操作。

In [ ]:
def my_function(x):
  if tf.reduce_sum(x) <= 1:
    return x * x
  else:
    return x-1

a_function = tf.function(my_function)

print("First branch, with graph:", a_function(tf.constant(1.0)).numpy())
print("Second branch, with graph:", a_function(tf.constant([5.0, 5.0])).numpy())

您可以直接调用Autograph转换来查看Python是如何转换为TensorFlow操作的。  这大部分是不可读的，但您可以看到转换。

In [ ]:
# 不要太在意输出
print(tf.autograph.to_code(my_function))

Autograph自动转换if-then子句、循环、break、return、continue等。

大多数的时候, Autograph工作的时候不会有特别的考虑。然而有些说明和[tf.function guide]以及[complete autograph reference]可以帮助到你。

### 看到速度的加快（Seeing the speed up）

在tf.function只是包装一个使用张量的函数并不能加速代码的运行，对于在一台机器上多次调用的小函数，调用一个图形或图形片段的开销可能会占用大多数运行时间。此外，如果大部分计算已经在加速器上进行，比如大量的gpu卷积，图形加速就不会很明显。

对于复杂的计算，图形可以提供显著的加速。这是因为图减少了python与设备的通信，并执行了一些加速操作。

这段代码在一些密集层上运行了几次，并统计了不同情况下的运行时间。

In [ ]:
# 分类图片的模型
class SequentialModel(tf.keras.Model):
  def __init__(self, **kwargs):
    super(SequentialModel, self).__init__(**kwargs)
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.dense_1 = tf.keras.layers.Dense(128, activation="relu")
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)

  def call(self, x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return x

input_data = tf.random.uniform([60, 28, 28])

eager_model = SequentialModel()
graph_model = tf.function(eager_model)

print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=10000))
print("Graph time:", timeit.timeit(lambda: graph_model(input_data), number=10000))


### 多态函数（Polymorphic functions）

跟踪一个函数时，将创建一个多态的**函数**对象。多态函数是一种Python可调用函数，它在一个API后面封装了几个具体的函数图。

您可以在所有不同类型的dtype和形状上使用此**函数**。每次使用新的参数签名调用它时，原始函数将使用新的参数重新跟踪。  然后该**函数**存储对应于**concrete_function**中跟踪的tf.graph。如果函数已经被这种参数跟踪过了，你就得到了你的预跟踪的图。

然后，从概念上讲:
* 一个**tf.Graph**是描述计算的原始的、可移植的数据结构
* **函数**对ConcreteFunctions进行缓存，跟踪和调度
* **ConcreteFunction**是一个与eager excution兼容的图包装器，它允许您从Python执行计算图



### 检查多态函数（Inspecting polymorphic functions）

您可以检查`a_function`，它是在Python函数的`my_function`中调用tf.function的结果。在本例中，使用三种参数调用`a_function`会产生三个不同的具体函数。

In [ ]:
print(a_function)

print("Calling a `Function`:")
print("Int:", a_function(tf.constant(2)))
print("Float:", a_function(tf.constant(2.0)))
print("Rank-1 tensor of floats", a_function(tf.constant([2.0, 2.0, 2.0])))

In [ ]:
# 获得工作在实数上的具体函数
print("Inspecting concrete functions")
print("Concrete function for float:")
print(a_function.get_concrete_function(tf.TensorSpec(shape=[], dtype=tf.float32)))
print("Concrete function for tensor of floats:")
print(a_function.get_concrete_function(tf.constant([2.0, 2.0, 2.0])))


In [ ]:
# 具体函数是可调用的
# 注意：你一般不需要这样做，你应调用包含的函数
cf = a_function.get_concrete_function(tf.constant(2))
print("Directly calling a concrete function:", cf(tf.constant(2)))

在这个示例中，您看到的是堆栈中相当深入的部分。除非您专门管理跟踪，否则通常不需要直接调用此处所示的具体函数

## 重新回到即刻执行（Reverting to eager execution）

您可能会发现自己正在查看长长的堆栈跟踪，特别是那些引用tf.Graph()或tf.Graph().as_default()的堆栈跟踪。这意味着您可能是在一个计算图上下文中运行。TensorFlow中的核心函数使用计算图上下文，比如Keras的model.fit()。

调试即刻执行通常要容易得多。堆栈跟踪应该相对简短并且容易理解。

在图使调试变得棘手的情况下，您可以恢复到使用即刻执行进行调试。

这里有一些方法，你可以确保你可以快速地运行:

* 直接将模型和层作为可调用对象调用

* 当使用Keras compile/fit时，在编译时使用model.compile(run_eagerly=True)

* 通过 tf.config.run_functions_eagerly(True)设置全局执行模式


### 使用`run_eagerly=True`

In [ ]:
# 定义一个恒等层，它具有eager副作用
class EagerLayer(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(EagerLayer, self).__init__(**kwargs)
    # Do some kind of initialization here

  def call(self, inputs):
    print("\nCurrently running eagerly", str(datetime.now()))
    return inputs

In [ ]:
# 创建一个新的模型对图像分类，它包含定制层
class SequentialModel(tf.keras.Model):
  def __init__(self):
    super(SequentialModel, self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.dense_1 = tf.keras.layers.Dense(128, activation="relu")
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)
    self.eager = EagerLayer()

  def call(self, x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return self.eager(x)

# 创建模型实例
model = SequentialModel()

# 产生一些无意义的图像和标签
input_data = tf.random.uniform([60, 28, 28])
labels = tf.random.uniform([60])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

首先，不采用即刻执行编译模型。注意，模型没有被跟踪;`compile`只设置损失函数、优化器和其他训练参数。

In [ ]:
model.compile(run_eagerly=False, loss=loss_fn)

现在，调用fit并看到该函数被跟踪(两次)，然后eager效果将不再运行。

In [ ]:
model.fit(input_data, labels, epochs=3)

即使你只即刻执行一轮，你也可以看到eage效果两次.

In [ ]:
print("Running eagerly")
# 使用即刻执行编码模型
model.compile(run_eagerly=True, loss=loss_fn)

model.fit(input_data, labels, epochs=1)


### 使用 `run_functions_eagerly`

你也可以在全局范围内设置每一项都可以即刻运行。这是一个绕过多态函数的跟踪函数并直接调用原始函数的开关。您可以使用它进行调试。

In [ ]:
# 现在，全局设置即刻执行
tf.config.run_functions_eagerly(True)
print("Run all functions eagerly.")

# 创建一个多态函数
polymorphic_function = tf.function(model)

print("Tracing")
# 这实际上在跟踪函数
print(polymorphic_function.get_concrete_function(input_data))

print("\nCalling twice eagerly")
# 当你再次运行函数时，由于函数即刻执行，你将看到2次效果
result = polymorphic_function(input_data)
result = polymorphic_function(input_data)

In [ ]:
# 别忘了关闭即刻执行
tf.config.experimental_run_functions_eagerly(False)

## 跟踪和性能（Tracing and performance）

跟踪会带来一些开销。虽然跟踪小功能很快，但大型模型可能需要明显的长时间来跟踪。这种投资通常会很快得到回报，带来性能提升，但要意识到，任何大型模型训练的前几个阶段都可能由于跟踪而变慢。

无论您的模型有多大，您都希望避免频繁跟踪。 这部分的[tf的函数指南](https://tensorflow.google.cn/guide/function#when_to_retrace)讨论了如何设置输入规格和使用张量参数来避免重跟踪retracing。 如果您发现性能异常差，最好检查一下是否不小心进行了重跟踪。

您可以添加一个eager-only的效果(比如打印Python参数)，这样就可以看到跟踪函数的时间。在这里，您可以看到额外的重跟踪，因为新的Python参数总是触发重跟踪。

In [ ]:
# 使用@tf.function装饰器
@tf.function
def a_function_with_python_side_effect(x):
  print("Tracing!")  # This eager
  return x * x + tf.constant(2)

# 第一次跟踪
print(a_function_with_python_side_effect(tf.constant(2)))

# The second time through, you won't see the side effect
print(a_function_with_python_side_effect(tf.constant(3)))

# This retraces each time the Python argument changes,
# as a Python argument could be an epoch count or other
# hyperparameter
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))
